In [ ]:
import lifelines
import pymc as pm
from pyBMA.CoxPH import CoxPH
import matplotlib.pyplot as plt
import numpy as np
from math import log
%matplotlib inline  

In [ ]:
#Extracting and formatting data

In [150]:
#####Put extracting and formatting here

In [ ]:
###Parametric Bayes
#Shout out to Cam Davidson-Pilon

In [ ]:
## Example fully worked model using toy data
## Adapted from http://blog.yhat.com/posts/estimating-user-lifetimes-with-pymc.html
## Note that there are corrections in the node to fix bugs in article

import pymc as mc
import numpy.ma as ma

N = 30
#create some artificial data, like the data in the figure above. 
lifetime = mc.rweibull( 2, 5, size = N ) 
birth = mc.runiform(0, 10, N)

censor = (birth + lifetime) > 10 #an individual is right-censored if this is True 
lifetime_ = ma.masked_array( lifetime, censor ) #create the censorship event. 
lifetime_.set_fill_value( 10 ) #good for computations later.

#this begins the model 
alpha = mc.Uniform("alpha", 0,20) 
#lets just use uninformative priors 
beta = mc.Uniform("beta", 0,20) 
obs = mc.Weibull( 'obs', alpha, beta, value = lifetime_, observed = True )

@mc.potential
def censorfactor(obs=obs): 
    if np.any((obs + birth < 10)[lifetime_.mask] ): 
        return -100000
    else:
        return 0

mcmc = mc.MCMC([alpha, beta, obs, censorfactor ] )
mcmc.sample(50000, burn = 10, thin = 1)

In [ ]:
pm.Matplot.plot(mcmc)

Problems: 

    1 - Work out the mean observed time to convert
    2 - Try to fit your data from section 1 
    3 - Try adjusting the number of samples, the burn parameter and the amount of thinning to correct get good answers
    4 - Try adjusting the prior and see how it affects the estimate
    5 - Try to fit a different distribution to the data
    6 - Compare answers
    
    

In [ ]:
#Solution to question 3:
### Increasing the burn parameter allows us to discard results before convergence
### Thinning the results removes autocorrelation

mcmc = mc.MCMC([alpha, beta, obs, censorfactor ] )
mcmc.sample(50000, burn = 30000, thin = 20)

pm.Matplot.plot(mcmc)

In [ ]:
#Solution to Q4
## Adjusting the priors impacts the overall result
## If we give a looser, less informative prior then we end up with a broader, shorter distribution
## If we give much more informative priors, then we get a tighter, taller distribution

import pymc as mc
import numpy.ma as ma

N = 30
#create some artificial data. 
lifetime = mc.rweibull( 2, 5, size = N ) 
birth = mc.runiform(0, 10, N)

censor = (birth + lifetime) > 10 
lifetime_ = ma.masked_array( lifetime, censor )
lifetime_.set_fill_value( 10 ) 

## Note the narrowing of the prior around the true values
alpha = mc.Uniform("alpha", 1, 3) 
beta = mc.Uniform("beta", 3, 6) 

####Uncomment this to see the result of looser priors
#alpha = mc.Uniform("alpha", 0, 30) 
#beta = mc.Uniform("beta", 0, 30) 

obs = mc.Weibull( 'obs', alpha, beta, value = lifetime_, observed = True )

@mc.potential
def censorfactor(obs=obs): 
    if np.any((obs + birth < 10)[lifetime_.mask] ): 
        return -100000
    else:
        return 0

mcmc = mc.MCMC([alpha, beta, obs, censorfactor ] )
mcmc.sample(50000, burn = 20000, thin = 20)
pm.Matplot.plot(mcmc)

In [ ]:
#Solution to Q5
## To fit a new distribution to the data, we need to change the obs variable to a new distribution.
## This requires creating new hyper parameters,
## The normal distribution has two parameters, the mean and the stdev (we use 1/stdev in pymc)

import pymc as mc
import numpy.ma as ma

N = 30
#create some artificial data. 
lifetime = mc.rweibull( 2, 5, size = N ) 
birth = mc.runiform(0, 10, N)

censor = (birth + lifetime) > 10 
lifetime_ = ma.masked_array( lifetime, censor )
lifetime_.set_fill_value( 10 ) 

#this begins the model 
alpha = mc.Uniform("mean", 0,15) 
tau = mc.Uniform("tau", 0, 10)
obs = mc.Normal( 'obs', alpha, tau, value = lifetime_, observed = True )

@mc.potential
def censorfactor(obs=obs): 
    if np.any((obs + birth < 10)[lifetime_.mask] ): 
        return -100000
    else:
        return 0

mcmc = mc.MCMC([alpha, beta, obs, censorfactor ] )
mcmc.sample(50000, burn = 10, thin = 1)
pm.Matplot.plot(mcmc)
sum(mcmc.trace("mean")[:])/ len(mcmc.trace("mean")[:])

In [ ]:
#Cox model

If we want to look at covariates, we need a new approach.  We'll use Cox proprtional hazards.  More information here.

In [ ]:
#Fitting solution
cf = lifelines.CoxPHFitter()
cf.fit(data, 'week', event_col = 'arrest')

Once we've fit the data, we need to do something useful with it.  Try to do the following things:

    1 - Plot the cumulative baseline hazard and survival function

            What relation do these have?

    2 - Predict the functions for a particular set of regressors

    3 - Plot the survival function for two different set of regressors

    4 - For your results in part 3 caculate how much more likely a death event is for one than the other for a given period of time

In [ ]:
#Solution to 1
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey = True)
cf.baseline_cumulative_hazard_.plot(ax = axes[0], title = "Baseline Cumulative Hazard")
cf.baseline_survival_.plot(ax = axes[1], title = "Baseline Survival")
print("Survival is the 1 - Cumulative hazard")

In [ ]:
# Solution to prediction
regressors = np.array([[0,0,0,0,0,0,0]])
survival = cf.predict_survival_function(regressors)

In [ ]:
#Solution to plotting multiple regressors
fig, axis = plt.subplots(nrows=1, ncols=1, sharex=True)
regressor1 = np.array([[0,0,0,0,0,0,0]])
regressor2 = np.array([[1,1,1,1,1,1,1]])
survival_1 = cf.predict_survival_function(regressor1)
survival_2 = cf.predict_survival_function(regressor2)
plt.plot(survival_1,label = "All zeros")
plt.plot(survival_2,label = "All ones")
plt.legend()

In [ ]:
#Difference in survival 
odds = survival_1 / survival_2
plt.plot(odds, c = "red")

Model selection

Difficult to do with classic tools (here)

Problem:

    1 - Calculate the BMA coefficient values

    2 - Compare these results to past the lifelines results

In [ ]:
bmaCox = CoxPH(data, "week", "arrest")
coefficient, se = bmaCox.run()

In [ ]:
print(coefficient)

In [ ]:
difference = coefficient - cf.summary["coef"]
print(difference)
print("Most very close, some significantly out (because no leaps and jumps)")